<a href="https://colab.research.google.com/github/aafreen2212/CreditRisk-DS/blob/master/HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [58]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [20]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

In [21]:
!pwd

/content


In [22]:
#Download the files
! kaggle competitions download -c home-credit-default-risk

 95% 103M/109M [00:00<00:00, 79.4MB/s] 
100% 109M/109M [00:01<00:00, 110MB/s] 
 95% 35.0M/36.8M [00:00<00:00, 52.9MB/s]
100% 36.8M/36.8M [00:00<00:00, 62.6MB/s]
 95% 258M/271M [00:02<00:00, 101MB/s] 
100% 271M/271M [00:02<00:00, 117MB/s]
 89% 32.0M/36.1M [00:00<00:00, 137MB/s]
100% 36.1M/36.1M [00:00<00:00, 139MB/s]
 86% 49.0M/56.8M [00:01<00:00, 33.2MB/s]
100% 56.8M/56.8M [00:01<00:00, 46.7MB/s]
 93% 90.0M/96.7M [00:00<00:00, 89.0MB/s]
100% 96.7M/96.7M [00:00<00:00, 102MB/s] 
 84% 64.0M/76.3M [00:00<00:00, 119MB/s] 
100% 76.3M/76.3M [00:00<00:00, 141MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
HomeCredit_columns_description.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/5.81M [00:00<?, ?B/s]
100% 5.81M/5.81M [00:00<00:00, 94.8MB/s]


In [23]:
#Unzip the files
!unzip \*.zip && rm *.zip

Archive:  application_train.csv.zip
replace application_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: application_train.csv   

Archive:  installments_payments.csv.zip
replace installments_payments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: installments_payments.csv  y
y


Archive:  bureau.csv.zip
replace bureau.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: bureau.csv              

Archive:  application_test.csv.zip
replace application_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: application_test.csv    

Archive:  credit_card_balance.csv.zip
replace credit_card_balance.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: credit_card_balance.csv  

Archive:  POS_CASH_balance.csv.zip
replace POS_CASH_balance.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: POS_CASH_balance.csv    y


Archive:  bureau_balance.csv.zip
replace bureau_balance.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: bureau_balance.csv  

In [69]:
application_train = pd.read_csv('application_train.csv')
application_test = pd.read_csv('application_test.csv')
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')
credit_card = pd.read_csv('credit_card_balance.csv')
cash_balance = pd.read_csv('POS_CASH_balance.csv')
installments = pd.read_csv('installments_payments.csv')
previous_application = pd.read_csv('previous_application.csv')

In [70]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
application_train.isnull().sum()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
71,100083,0.0,Cash loans,M,Y,Y,0,103500.0,573628.5,24435.0,463500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.009657,-15406,-892,-341.0,-4176,22.0,1,1,0,1,0,0,Laborers,2.0,2,2,MONDAY,18,0,0,0,0,0,0,...,0.1214,0.0949,0.0000,0.0000,reg oper account,block of flats,0.1324,"Stone, brick",No,2.0,1.0,2.0,1.0,-2053.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
124,100145,0.0,Cash loans,F,Y,Y,1,202500.0,260725.5,16789.5,198000.0,Family,Working,Secondary / secondary special,Separated,House / apartment,0.018850,-16282,-4375,-762.0,-1494,8.0,1,1,0,1,0,0,Laborers,2.0,2,2,TUESDAY,11,0,0,0,0,0,0,...,0.0770,0.0919,0.0000,0.0000,reg oper spec account,block of flats,0.0710,Panel,No,6.0,0.0,6.0,0.0,-1782.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
152,100179,0.0,Cash loans,F,Y,N,0,202500.0,675000.0,53329.5,675000.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.031329,-11375,-2311,-180.0,-2009,4.0,1,1,0,1,0,0,Managers,2.0,2,2,MONDAY,13,0,0,0,0,0,0,...,0.4361,0.7317,0.2213,0.7892,reg oper account,block of flats,0.7334,Monolithic,No,0.0,0.0,0.0,0.0,-1792.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
161,100190,0.0,Cash loans,M,Y,N,0,162000.0,263686.5,24781.5,238500.0,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,0.022625,-13972,-4472,-464.0,-4529,3.0,1,1,0,1,1,0,Laborers,2.0,2,2,THURSDAY,16,0,0,0,0,0,0,...,0.2565,0.3628,0.0000,0.0172,reg oper account,block of flats,0.4064,Panel,No,5.0,0.0,5.0,0.0,-1161.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
255,100295,1.0,Cash loans,M,Y,N,1,225000.0,1019205.0,31032.0,774000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0.072508,-11356,-602,-335.0,-3224,9.0,1,1,0,1,0,0,Laborers,2.0,1,1,MONDAY,14,1,0,1,0,0,0,...,0.1975,0.2310,0.0078,0.0077,reg oper account,block of flats,0.1874,Panel,No,3.0,0.0,3.0,0.0,-14.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,6.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307358,456083,0.0,Cash loans,F,Y,Y,2,112500.0,361462.5,16051.5,274500.0,Unaccompanied,Commercial associate,Secondary /

In [65]:
# Number of unique classes in each object column
application_data.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [74]:
pd.unique(application_train.CODE_GENDER)

array(['M', 'F', 'XNA'], dtype=object)

In [75]:
application_train['CODE_GENDER'].replace('XNA', np.nan, inplace=True)

In [90]:
application_data = application_train.append(application_test)

In [91]:
(application_data.isna().sum())

SK_ID_CURR                        0
TARGET                        48744
NAME_CONTRACT_TYPE                0
CODE_GENDER                       4
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     47568
AMT_REQ_CREDIT_BUREAU_WEEK    47568
AMT_REQ_CREDIT_BUREAU_MON     47568
AMT_REQ_CREDIT_BUREAU_QRT     47568
AMT_REQ_CREDIT_BUREAU_YEAR    47568
Length: 122, dtype: int64

In [94]:
print(len(application_data.select_dtypes('object').columns))
objects = application_data.select_dtypes('object').columns
application_data[objects] = application_data[objects].fillna('Unknown')
application_data.select_dtypes('object').columns

16


Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [95]:
print(len(application_data.select_dtypes('float64').columns))
float64 = application_data.select_dtypes('float64').columns[1:]
application_data[float64] = application_data[float64].fillna(application_data[float64].mean())
application_data.select_dtypes('float64').columns

66


Index(['TARGET', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_REGISTRATION',
       'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1', 'EXT_SOURCE_2',
       'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG',
       'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG',
       'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG',
       'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE',
       'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE',
       'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE',
       'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI',
       'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVA

In [96]:
print(len(application_data.select_dtypes('int64').columns))
int64 = application_data.select_dtypes('int64').columns
application_data[int64] = application_data[int64].fillna(application_data[int64].mean())
application_data.select_dtypes('int64').columns

40


Index(['SK_ID_CURR', 'CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
       'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
       'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21'],
      dtype='object')

In [99]:
application_data.isnull().sum().sum(), application_data.shape

(48744, (356255, 122))

In [101]:
application_data = pd.get_dummies(application_data)
application_data.shape

(356255, 252)

Bureau Data

In [52]:
bureau_balance.isnull().sum()

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [55]:
def remove_na(df1, df2):
  missing_data = df1.isnull().sum().sort_values(ascending=False)
  missing_percent = (100* missing_data) / df1.shape[0]
  df_missing = pd.concat([missing_data,missing_percent], axis = 1, keys=['Total_Missing_Data', 'Percentage'])
  df_missing.index.name = 'Fields'
  fields = df_missing[df_missing['Percentage']<70].index
  df = df1[fields]
  df2 = df2[df1.columns]
  return df1, df2

In [103]:
combine = application_data

In [104]:
X = combine[combine['TARGET'].isnull()!=True].drop(['TARGET', 'SK_ID_CURR'], axis=1)
y = combine[combine['TARGET'].isnull()!=True]['TARGET'].reset_index(drop=True)

X_test = combine[combine['TARGET'].isnull()==True].drop(['TARGET','SK_ID_CURR'], axis=1)

X.shape, y.shape, X_test.shape

((307511, 250), (307511,), (48744, 250))

In [ ]:
err_as = []
err_rs = []
err_ps = []
err_roc = []
err_ll = []

y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, roc_auc_score, log_loss
from sklearn.ensemble import RandomForestClassifier

fold = StratifiedKFold(n_splits=5)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = RandomForestClassifier()
    m.fit(x_train, y_train)
    pred_y = m.predict(x_val)
    prob_pred = m.predict_proba(x_val)[:,1]
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print("Fold ",i, " Accuracy: ",(accuracy_score(pred_y, y_val)))
      print("Fold ",i, " Recall: ",(recall_score(pred_y, y_val)))
    print("Fold ",i, " Precision: ",(precision_score(pred_y, y_val)))
    print("Fold ",i, " ROC AUC: ",(roc_auc_score(y_val, prob_pred)))
    print("Fold ",i, " Logloss: ",(log_loss(y_val, prob_pred)))
    print(confusion_matrix(pred_y, y_val))

    err_as.append(accuracy_score(pred_y, y_val))
    err_rs.append(recall_score(pred_y, y_val))
    err_ps.append(precision_score(pred_y, y_val))
    err_roc.append(roc_auc_score(y_val, prob_pred))
    err_ll.append(log_loss(y_val, prob_pred))

    pred_test = m.predict_proba(X_test)[:,1]
    i = i + 1

In [ ]:
print('Mean Accuracy Score on CV-5: ', np.mean(err_as, 0))
print('Mean Precision Score on CV-5: ', np.mean(err_ps, 0))
print('Mean Recall Score on CV-5: ', np.mean(err_rs, 0))
print('Mean ROC AUC Score on CV-5: ', np.mean(err_roc, 0))
print('Mean Logloss Score on CV-5: ', np.mean(err_ll, 0))

In [ ]:

all_feat = feature_importance_df[["Feature",
                                  "importance"]].groupby("Feature").mean().sort_values(by="importance", 
                                                                                           ascending=False)
all_feat.reset_index(inplace=True)
important_feat = list(all_feat['Feature'])
all_feat.head(20)

In [ ]:
df = X[important_feat]
corr_matrix = df.corr().abs()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

high_cor = [column for column in upper.columns if any(upper[column] > 0.98)]
print(len(high_cor))
print(high_cor)

In [ ]:
err_as_lreg = []
err_rs_lreg = []
err_ps_lreg = []
err_roc_lreg = []
err_ll_lreg = []

y_pred_tot_logreg = []

from sklearn.linear_model import LogisticRegression

fold = StratifiedKFold(n_splits=5)
i = 1
for train_index, test_index in fold.split(X1, y):
    x_train, x_val = X1.iloc[train_index], X1.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LogisticRegression()
    m.fit(x_train, y_train)
    pred_y = m.predict(x_val)
    prob_pred = m.predict_proba(x_val)[:,1]
    
    print("Fold ",i, " Accuracy: ",(accuracy_score(pred_y, y_val)))
    print("Fold ",i, " Recall: ",(recall_score(pred_y, y_val)))
    print("Fold ",i, " Precision: ",(precision_score(pred_y, y_val)))
    print("Fold ",i, " ROC AUC: ",(roc_auc_score(y_val, prob_pred)))
    print("Fold ",i, " Logloss: ",(log_loss(y_val, prob_pred)))
    print(confusion_matrix(pred_y, y_val))
    err_as_lreg.append(accuracy_score(pred_y, y_val))
    err_rs_lreg.append(recall_score(pred_y, y_val))
    err_ps_lreg.append(precision_score(pred_y, y_val))
    err_roc_lreg.append(roc_auc_score(y_val, prob_pred))
    err_ll_lreg.append(log_loss(y_val, prob_pred))

    pred_test = m.predict_proba(X_test1)[:,1]
    i = i + 1
    y_pred_tot_logreg.append(pred_test

In [ ]:

print('Mean Accuracy Score on CV-5: ', np.mean(err_as_lreg, 0))
print('Mean Precision Score on CV-5: ', np.mean(err_ps_lreg, 0))
print('Mean Recall Score on CV-5: ', np.mean(err_rs_lreg, 0))
print('Mean ROC AUC Score on CV-5: ', np.mean(err_roc_lreg, 0))
print('Mean Logloss Score on CV-5: ', np.mean(err_ll_lreg, 0)